<a href="https://colab.research.google.com/github/harnalashok/deeplearning-sequences/blob/main/chatbot_withg_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.linkedin.com/pulse/get-insight-from-your-business-data-build-llm-application-jain/

In [ ]:
!git lfs install
!git clone https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
!git clone https://huggingface.co/google/flan-t5-large

In [ ]:
!pip install langchain
!pip install torch
!pip install transformers
!pip install faiss-cpu
!pip install pypdf
!pip install sentence-transformers

In [14]:
from langchain.document_loaders import PyPDFLoader
pdfLoader = PyPDFLoader("/content/sample_data/large_language_models.pdf")
documents = pdfLoader.load()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
modelPath = "/content/all-MiniLM-L6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

In [7]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [8]:
question = "What are the variants of the transformer architectures used in LLMs?"
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

where Xis the input of layer and l,W, b, V andcare learned
parameters.
Other GLU variants [74] used in LLMs are:
ReGLU (x, W, V, b, c ) =max(0, xW +b)⊗,
GEGLU (x, W, V, b, c ) =GELU (xW+b)⊗(xV+c),
SwiGLU (x, W, V, b, c, β ) =Swishβ (xW+b)⊗(xV+c).
E. Layer Normalization
Layer normalization leads to faster convergence and is an
integrated component of transformers [64]. Besides Layer-
Norm [76] and RMSNorm [77], LLMs use pre-layer normal-
ization [78], applying it before multi-head attention (MHA).
Pre-norm is shown to provide training stability in LLMs.
Another normalization variant, DeepNorm [79] fixes larger
gradient issue in pre-norm.
F . Distributed LLM Training
This section describes distributed LLM training approaches
briefly. More details are available in [13], [37], [80], [81].
Data Parallelism: Data parallelism replicates the model on
multiple devices where data in a batch gets divided across


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,pipeline
from langchain import HuggingFacePipeline

tokenizer = AutoTokenizer.from_pretrained("/content/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("/content/flan-t5-large")
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0, "max_length": 1512, "max_new_tokens" : 1500},
)

In [10]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [11]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=db.as_retriever(),
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [12]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. \n{context}\nQuestion: {question}\nHelpful Answer:"), llm=HuggingFacePipeline(pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x7b8bd506dc30>, model_kwargs={'temperature': 0, 'max_length': 1512, 'max_new_tokens': 1500})), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7b8c1da48460>))

In [13]:
result = qa_chain({ "query" : question })
print(result["result"])


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Token indices sequence length is longer than the specified maximum sequence length for this model (1038 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


The difference arises due to the application of the attention and the connection of transformer blocks.
